In [1]:
import pandas as pd
from outrigger.region import Region
from outrigger.io.gtf import SplicingAnnotator

import gffutils

%load_ext autoreload
%autoreload 2 

In [2]:
import outrigger

In [3]:
outrigger.__file__

'/home/obotvinnik/workspace-git/outrigger/outrigger/__init__.pyc'

In [4]:
mxe_events = pd.read_csv('/home/obotvinnik/projects/singlecell_pnms/analysis/outrigger_v2/index/mxe/events.csv', index_col=0)
print(mxe_events.shape)
mxe_events.head(2)

(325347, 5)


,junction12,junction13,junction24,junction34,incompatible_junctions
event_id,,,,,
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212300:+@junction:chr17:75212301-75212326:+|isoform2=junction:chr17:75212018-75212131:+@novel_exon:chr17:75212132-75212144:+@junction:chr17:75212145-75212326:+,junction:chr17:75212018-75212131:+,junction:chr17:75212018-75212222:+,junction:chr17:75212145-75212326:+,junction:chr17:75212301-75212326:+,junction:chr17:75212018-75212326:+|junction:ch...
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212287:+@junction:chr17:75212288-75212326:+|isoform2=junction:chr17:75212018-75212079:+@novel_exon:chr17:75212080-75212144:+@junction:chr17:75212145-75212326:+,junction:chr17:75212018-75212079:+,junction:chr17:75212018-75212222:+,junction:chr17:75212145-75212326:+,junction:chr17:75212288-75212326:+,junction:chr17:75212018-75212326:+|junction:ch...


In [5]:
# mxe_events_not_duplicated = mxe_events.drop_duplicates()
# print(mxe_events_not_duplicated.shape)
# mxe_events_not_duplicated.head(2)

(18257, 5)


,junction12,junction13,junction24,junction34,incompatible_junctions
event_id,,,,,
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212300:+@junction:chr17:75212301-75212326:+|isoform2=junction:chr17:75212018-75212131:+@novel_exon:chr17:75212132-75212144:+@junction:chr17:75212145-75212326:+,junction:chr17:75212018-75212131:+,junction:chr17:75212018-75212222:+,junction:chr17:75212145-75212326:+,junction:chr17:75212301-75212326:+,junction:chr17:75212018-75212326:+|junction:ch...
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212287:+@junction:chr17:75212288-75212326:+|isoform2=junction:chr17:75212018-75212079:+@novel_exon:chr17:75212080-75212144:+@junction:chr17:75212145-75212326:+,junction:chr17:75212018-75212079:+,junction:chr17:75212018-75212222:+,junction:chr17:75212145-75212326:+,junction:chr17:75212288-75212326:+,junction:chr17:75212018-75212326:+|junction:ch...


In [5]:
%%time
mxe_exon_bed_template = '/home/obotvinnik/projects/singlecell_pnms/analysis/outrigger_v2/index/mxe/exon{}.bed'

exons = {}

for i in range(1, 5):
    exon = 'exon{}'.format(i)
    bed = mxe_exon_bed_template.format(i)
    exon_df = pd.read_table(bed, names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
    exon_regions = exon_df.apply(lambda row: Region('exon:{chrom}:{start}-{stop}:{strand}'.format(
                chrom=row.chrom, start=row.start+1, stop=row.stop, strand=row.strand)), axis=1)
    exons[exon] = exon_regions
exon_regions_df = pd.DataFrame(exons)
print(exon_regions_df.shape)
exon_regions_df.head()

(325347, 4)
CPU times: user 2min 43s, sys: 913 ms, total: 2min 44s
Wall time: 2min 45s


In [6]:
exon_str = exon_regions_df.applymap(lambda x: x.name)
print(exon_str.shape)
exon_str.head()

(325347, 4)


,exon1,exon2,exon3,exon4
0,exon:chr17:75211927-75212017:+,exon:chr17:75212132-75212144:+,exon:chr17:75212223-75212300:+,exon:chr17:75212327-75212352:+
1,exon:chr17:75211940-75212017:+,exon:chr17:75212080-75212144:+,exon:chr17:75212223-75212287:+,exon:chr17:75212327-75212417:+
2,exon:chr17:75212005-75212079:+,exon:chr17:75212171-75212209:+,exon:chr17:75212210-75212300:+,exon:chr17:75212379-75212391:+
3,exon:chr17:75211979-75212017:+,exon:chr17:75212132-75212157:+,exon:chr17:75212158-75212222:+,exon:chr17:75212275-75212300:+
4,exon:chr5:180670692-180670872:-,exon:chr5:180669170-180669234:-,exon:chr5:180668492-180668639:-,exon:chr5:180666517-180666582:-


In [7]:
exon_str_not_duplicated = exon_str.drop_duplicates()
print(exon_str_not_duplicated.shape)
exon_str_not_duplicated.head()

(325347, 4)


,exon1,exon2,exon3,exon4
0,exon:chr17:75211927-75212017:+,exon:chr17:75212132-75212144:+,exon:chr17:75212223-75212300:+,exon:chr17:75212327-75212352:+
1,exon:chr17:75211940-75212017:+,exon:chr17:75212080-75212144:+,exon:chr17:75212223-75212287:+,exon:chr17:75212327-75212417:+
2,exon:chr17:75212005-75212079:+,exon:chr17:75212171-75212209:+,exon:chr17:75212210-75212300:+,exon:chr17:75212379-75212391:+
3,exon:chr17:75211979-75212017:+,exon:chr17:75212132-75212157:+,exon:chr17:75212158-75212222:+,exon:chr17:75212275-75212300:+
4,exon:chr5:180670692-180670872:-,exon:chr5:180669170-180669234:-,exon:chr5:180668492-180668639:-,exon:chr5:180666517-180666582:-


In [8]:
exon_str.index = mxe_events.index
print(exon_str.shape)
exon_str.head(2)

(325347, 4)


,exon1,exon2,exon3,exon4
event_id,,,,
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212300:+@junction:chr17:75212301-75212326:+|isoform2=junction:chr17:75212018-75212131:+@novel_exon:chr17:75212132-75212144:+@junction:chr17:75212145-75212326:+,exon:chr17:75211927-75212017:+,exon:chr17:75212132-75212144:+,exon:chr17:75212223-75212300:+,exon:chr17:75212327-75212352:+
isoform1=junction:chr17:75212018-75212222:+@novel_exon:chr17:75212223-75212287:+@junction:chr17:75212288-75212326:+|isoform2=junction:chr17:75212018-75212079:+@novel_exon:chr17:75212080-75212144:+@junction:chr17:75212145-75212326:+,exon:chr17:75211940-75212017:+,exon:chr17:75212080-75212144:+,exon:chr17:75212223-75212287:+,exon:chr17:75212327-75212417:+


## Use outrigger's `SplicingAnnotator`


In [9]:

outrigger_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_v2'

db_filename = '{}/index/gtf/gencode.v19.annotation.gtf.db'.format(outrigger_folder)

db = gffutils.FeatureDB(db_filename)

In [10]:
splicing_annotator = SplicingAnnotator(db, events=exon_str, splice_type='mxe')

In [12]:
range(3)

[0, 1, 2]

In [13]:
3+3

6

In [ ]:
attributes = splicing_annotator.attributes()

In [ ]:
list(db.region('chr17:75211927-75212017', featuretype='exon'))

In [ ]:
list(db.region('chr17:75211927-75212017', featuretype='gene'))